# IMPORT DEPENDENCIES

In [1]:
# Add the parent directory of the current script to the system path
# to enable importing modules from that directory or its subdirectories.

import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
# Import necessary modules and functions
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import pandas as pd
import geopandas as gpd
# Import custom modules and functions
from src.processing import read_outlets, join_watersheds2points, load_river_network, clip_river_network, insert_watershed_info, process_watershed_points
from src.snap_pour_point import read_flow_accumulation_tif, calculate_new_pour_point
from src.delineator import read_drainage_direction, calculate_upstream_v2  
from src.polygonize import raster_to_polygon, rasterize_array

from run_config_partial import OUTLETS, WATERSHEDS, MODE, FLOW_ACCUMULATION, DRAINAGE_DIRECTION, PIXEL2SEARCH, RIVERS, RESULTS, MAX_STRAHLER


In [21]:
points = read_outlets(OUTLETS)
points_copy = points.copy()
if MODE == "single":

    accum, pixel_size = read_flow_accumulation_tif(FLOW_ACCUMULATION)

    drainage_direction, tif_profile, dr_dir_src = read_drainage_direction(DRAINAGE_DIRECTION)

    river_vector = load_river_network(RIVERS)   
    
    points_copy = process_watershed_points(points, accum, pixel_size, drainage_direction, dr_dir_src,
                            tif_profile, river_vector, MAX_STRAHLER, RESULTS)
elif MODE == "partial":

    points_labelled = join_watersheds2points(points, WATERSHEDS)
    unique_watershed_ids = points_labelled["Watershed_ID"].unique()

    for watershed in unique_watershed_ids:
        print(f"{watershed}")
        filtered_points_labelled  = points_labelled[points_labelled["Watershed_ID"] == watershed]    
        accum, pixel_size = read_flow_accumulation_tif(os.path.join(FLOW_ACCUMULATION, watershed + '.tif'))

        drainage_direction, tif_profile, dr_dir_src = read_drainage_direction(os.path.join(DRAINAGE_DIRECTION, watershed + '.tif'))

        river_vector = load_river_network(os.path.join(RIVERS, watershed + '.geojson'))   
        
        points_copy = process_watershed_points(filtered_points_labelled, accum, pixel_size, drainage_direction, dr_dir_src,
                                tif_profile, river_vector, MAX_STRAHLER, RESULTS) 
        points_labelled = points_labelled.merge(points_copy[['id', 'snap_long', 'snap_lat', 'CalculatedArea[km2]', 'status', 'comment']], 
                          how='left', left_on=['id'], right_on=['id'])


TR03
[+] Processing 1.
TR12


RasterioIOError: ../data/raster/flow_accumulation/TR12.tif: No such file or directory

In [22]:
points_labelled

,id,name,long,lat,area[km2],index_right,Watershed_ID,Watershed_Name,Watershed_Area,snap_long,snap_lat,CalculatedArea[km2],status,comment
0,1,point_1,29.46242,39.60424,1000,2,TR03,Susurluk,24307.12,29.4625,39.605,1604.9,success,
1,4,point_4,29.47600,40.05800,100,11,TR12,Sakarya,63317.48,NaN,NaN,NaN,NaN,NaN


In [14]:
points_copy[["id", "snap_long", "snap_lat", "CalculatedArea[km2]", "status", "comment"]]

,id,snap_long,snap_lat,CalculatedArea[km2],status,comment
0,1,29.4625,39.605,1604.9,success,


In [11]:
points_labelled

,id,name,long,lat,area[km2],index_right,Watershed_ID,Watershed_Name,Watershed_Area
0,1,point_1,29.46242,39.60424,1000,2,TR03,Susurluk,24307.12
1,4,point_4,29.47600,40.05800,100,11,TR12,Sakarya,63317.48


In [18]:
points_labelled = points_labelled.merge(points_copy[['id', 'snap_long', 'snap_lat', 'CalculatedArea[km2]', 'status', 'comment']], 
                          how='left', left_on=['id'], right_on=['id'])

In [20]:
points_labelled

,id,name,long,lat,area[km2],index_right,Watershed_ID,Watershed_Name,Watershed_Area
0,1,point_1,29.46242,39.60424,1000,2,TR03,Susurluk,24307.12
1,4,point_4,29.47600,40.05800,100,11,TR12,Sakarya,63317.48


In [20]:
_MODE = "partial"
if MODE == "partial":
    points_labelled = join_watersheds2points(points, WATERSHEDS)
    unique_watershed_ids = points_labelled["Watershed_ID"].unique()
    for watershed in unique_watershed_ids[:1]:
        filtered_points_labelled  = points_labelled[points_labelled["Watershed_ID"] == watershed] 
    

In [23]:
watershed

'TR03'

In [4]:
for index, row in points.iterrows():
    print(f"[+] Proccessing {row.id}.")
    # Calculate new pour point
    new_pour_point = calculate_new_pour_point(accum, pixel_size, (row.long, row.lat), PIXEL2SEARCH)
    new_pour_point_xy = dr_dir_src.index(new_pour_point[0], new_pour_point[1])
    # Extract watersheds
    upstream_area = calculate_upstream_v2(drainage_direction, new_pour_point_xy)
    rasterized_array = rasterize_array(upstream_area, tif_profile)
    # Save polygon and line as JSON
    subbasin = raster_to_polygon(rasterized_array, save_polygon=True, polygon_save_path=os.path.join(RESULTS,"watershed", str(row.id)+"_basin"))
    # Clip rivers

    clipped_river_network, feedback = clip_river_network(river_vector, subbasin, max_strahler_order = MAX_STRAHLER, line_save_path=os.path.join(RESULTS,"river", str(row.id)+"_river"))

    # Insert watershed delienation information into the points table
    points_copy = insert_watershed_info(points_copy, row, new_pour_point, subbasin["CalculatedArea[km2]"][0], feedback)


[+] Proccessing 1.
[+] Proccessing 4.


In [15]:
process_watershed_points(points, accum, pixel_size, drainage_direction, dr_dir_src,
                            tif_profile, river_vector, MAX_STRAHLER, RESULTS)

[+] Processing 1.
[+] Processing 4.


,id,name,long,lat,area[km2],geometry,snap_long,snap_lat,CalculatedArea[km2],status,comment
0,1,point_1,29.46242,39.60424,1000,POINT (29.46242 39.60424),29.4625,39.605000,1604.9,success,
1,4,point_4,29.47600,40.05800,100,POINT (29.47600 40.05800),29.4750,40.059167,1.2,fail,no rivers clipped within the given basin.


In [12]:
type(dr_dir_src)

rasterio.io.DatasetReader

In [ ]:
points

In [ ]:
def loop_over_points():
    for index, row in points.iterrows():
   
        # Calculate new pour point
        new_pour_point = calculate_new_pour_point(accum, pixel_size, (row.long, row.lat), PIXEL2SEARCH)
        new_pour_point_xy = dr_dir_src.index(new_pour_point[0], new_pour_point[1])
        # Extract watersheds
        upstream_area = calculate_upstream_v2(drainage_direction, new_pour_point_xy)
        rasterized_array = rasterize_array(upstream_area, tif_profile)
        # Save polygon and line as JSON
        subbasin = raster_to_polygon(rasterized_array, save_polygon=True, polygon_save_path=os.path.join(RESULTS,"watershed", str(row.id)+"_basin"))
        # Clip rivers
        clipped_river_network = clip_river_network(river_vector, subbasin, max_strahler_order = MAX_STRAHLER, line_save_path=os.path.join(RESULTS,"river", str(row.id)+"_river"))


In [ ]:

for index, row in points.iterrows():
   
    # Calculate new pour point
    new_pour_point = calculate_new_pour_point(accum, pixel_size, (row.long, row.lat), PIXEL2SEARCH)
    new_pour_point_xy = dr_dir_src.index(new_pour_point[0], new_pour_point[1])
    # Extract watersheds
    upstream_area = calculate_upstream_v2(drainage_direction, new_pour_point_xy)
    rasterized_array = rasterize_array(upstream_area, tif_profile)
    # Save polygon and line as JSON
    subbasin = raster_to_polygon(rasterized_array, save_polygon=True, polygon_save_path=os.path.join(RESULTS,"watershed", str(row.id)+"_basin"))
    # Clip rivers
    clipped_river_network = clip_river_network(river_vector, subbasin, max_strahler_order = MAX_STRAHLER, line_save_path=os.path.join(RESULTS,"river", str(row.id)+"_river"))
      

In [ ]:
def read_drainage_direction(drainage_direction_path, outlets=None, pour_point_coord=None):
    """
    Reads the drainage direction data from a TIFF file.

    Parameters:
        drainage_direction_path (str): File path of the drainage direction TIFF.
        outlets (geopandas.GeoDataFrame, optional): GeoDataFrame containing outlets with 'long' and 'lat' columns.
            Each outlet's coordinates will be used as pour points to locate specific cells in the TIFF.
        pour_point_coord (tuple, optional): Coordinates of the pour point in the format (x, y).
            The pour point coordinates are used to locate the specific cell in the TIFF.

    Returns:
        tuple: A tuple containing the drainage direction data as a NumPy array,
               the pour point coordinates in the TIFF, and the metadata profile of the TIFF.

    Notes:
        - The function uses the rasterio library to read the TIFF file.
        - The drainage direction data is typically represented as an array where each
          cell represents the direction of flow.
        - If `pour_point_coord` is provided, it is used as the pour point to locate the specific cell in the TIFF.
        - If `outlets` is provided, each outlet's coordinates are used as pour points to locate specific cells in the TIFF.
        - The metadata profile contains information about the TIFF such as its spatial
          reference system, resolution, and other properties.

    Raises:
        ValueError: If both `outlets` and `pour_point_coord` are provided at the same time, or if `outlets` is not a GeoDataFrame.
    """
    if outlets is not None and pour_point_coord is not None:
        raise ValueError("Only one of 'outlets' or 'pour_point_coord' should be provided, not both.")
    if outlets is not None and (outlets.empty or outlets is None):
        raise ValueError("'outlets' GeoDataFrame is empty or None.")
    if outlets is not None and not isinstance(outlets, gpd.GeoDataFrame):
        raise ValueError("'outlets' must be a GeoDataFrame.")

    with rasterio.open(drainage_direction_path) as src:
        drainage_direction = src.read(1)
        profile = src.profile
        if pour_point_coord:
            pour_point_xy = src.index(pour_point_coord[0], pour_point_coord[1])
            return drainage_direction, pour_point_xy, profile
        if outlets is not None:
            for index, row in outlets.iterrows():
                pour_point_coord = row.long, row.lat
                pour_point_xy = src.index(pour_point_coord[0], pour_point_coord[1])
                outlets.loc[index, 'pour_point_x'] = int(pour_point_xy[0])
                outlets.loc[index, 'pour_point_y'] = int(pour_point_xy[1])
            return drainage_direction, outlets, profile

# SET THE PARAMETERS

In [ ]:
# Number of neighboring pixels to consider 
# in search of the neigboring pixel with the highest flow accumualtion
PIXEL2SEARCH = 1

# CALCULATION

In [ ]:
path = '../data/flow_accumulation_TR.tif'
coord = (28.864, 40.130) 
coord = (28.963, 40.118) 

data, pixel_size = read_flow_accumulation_tif(path)
new_pour_point = calculate_new_pour_point(data, pixel_size, coord, PIXEL2SEARCH)
print(new_pour_point)

In [ ]:
drainage_direction_path = "../data/drainage_direction_TR.tif"
dr_dir, tif_profile, dr_dir_src = read_drainage_direction(drainage_direction_path)
pour_point_xy  = dr_dir_src.index(new_pour_point[0, new_pour_point[1]])
# Calculate upstream area
upstream_area = calculate_upstream_v2(dr_dir, pour_point_xy)

rasterized_array = rasterize_array(upstream_area, tif_profile, )
subbasin = raster_to_polygon(rasterized_array, save_polygon=False, polygon_save_path="output/polygon")

In [ ]:
subbasin.plot()

In [ ]:
watershed_id = "TR03"

In [ ]:
river_vector = load_river_network(watershed_id) 

In [ ]:
clipped_river_network = clip_river_network(river_vector, subbasin, 'output/river.geojson')

# TEST

## Read points

In [ ]:
points = read_outlets(OUTLETS)

In [ ]:
points.plot()

In [ ]:
points_labelled = join_watersheds2points(points, WATERSHEDS)
unique_watershed_ids = points_labelled["Watershed_ID"].unique()
for watershed in unique_watershed_ids[:1]:
    filtered_points_labelled  = points_labelled[points_labelled["Watershed_ID"] == watershed] 

In [ ]:
for watershed in unique_watershed_ids[:1]:
    filtered_points_labelled  = points_labelled[points_labelled["Watershed_ID"] == watershed] 

In [ ]:
filtered_points_labelled

In [ ]:
for index, row in filtered_points_labelled.iterrows():
    coord = [row["long"], row["lat"]]
    
    
    

In [ ]:
points = read_outlets(OUTLETS)

# if MODE == "single":
    